In [61]:
!pip install ortools==9.7.2996 pandas==2.1.0

In [62]:
from ortools.sat.python import cp_model

In [63]:
variaveis_a_logar = []

# variavel global do genero nao eh boa pratica, usando somente para logar e nos informar
# nao colocar em codigo para producao
def loga(variavel):
  global variaveis_a_logar
  variaveis_a_logar.append(variavel)

In [64]:
def resolve(solucionador, modelo, estacionamentos, avioes, penalidades):
  global variaveis_a_logar

  modelo.Minimize(sum(penalidades))
  status = solucionador.Solve(modelo)
  print(solucionador.StatusName(status))
  if status == cp_model.INFEASIBLE:
    print("Sem solucao")
    return

  print(f"Valor objetivo = {solucionador.ObjectiveValue()}")

  for variavel in variaveis_a_logar:
    print(f"{variavel} tem valor {solucionador.Value(variavel)}")

  for estacionamento in estacionamentos:
    variavel = estacionamento.variavel
    valor = solucionador.Value(variavel)
    if valor == 0:
      print(f"{variavel} sem aviao")
    else:
      aviao = avioes[valor - 1]
      print(f"{variavel} tem aviao {valor} grande={aviao.grande}")


In [65]:
def avioes_distintos(estacionamentos, modelo):
  variaveis = [estacionamento.variavel for estacionamento in estacionamentos]
  modelo.AddAllDifferent(variaveis)

In [66]:
def todo_aviao_tem_que_estacionar(total_de_avioes, estacionamentos, modelo):
  variaveis = {}
  for i in range(1, total_de_avioes + 1):
    for j, estacionamento in enumerate(estacionamentos):
      aviao_i_em_j = modelo.NewBoolVar(f'aviao_{i}_em{j}')
      modelo.Add(estacionamento.variavel == i).OnlyEnforceIf(aviao_i_em_j)
      modelo.Add(estacionamento.variavel != i).OnlyEnforceIf(aviao_i_em_j.Not())
      variaveis[(i, j)] = aviao_i_em_j

  for i in range(1, total_de_avioes + 1):
    modelo.AddExactlyOne([variaveis[(i,j)] for j in range(len(estacionamentos))])


In [67]:
class Estacionamento:
  def __init__(self, k, total_de_avioes, grande, modelo, tem_controle_de_passaporte):
    self.grande = grande
    self.tem_controle_de_passaporte = tem_controle_de_passaporte
    self.variavel = modelo.NewIntVar(0, total_de_avioes, f'estacionamento_{k}')
    self.k = k
    self.vizinhos = []
    self.recebe_aviao_grande = modelo.NewBoolVar(f'recebe_aviao_grande_{k}')
    if not self.grande:
      modelo.Add(self.recebe_aviao_grande == 0)

In [68]:
class Aviao:
  def __init__(self, k, grande, requer_controle_de_passaporte):
    self.k = k
    self.grande = grande
    self.requer_controle_de_passaporte = requer_controle_de_passaporte

In [69]:
def limita_aviao_grande_para_estacionamento_grande(modelo, estacionamentos, avioes):
  avioes_grandes = [aviao for aviao in avioes if aviao.grande]
  for estacionamento in estacionamentos:
    for aviao in avioes_grandes:
      modelo.Add(estacionamento.variavel != aviao.k).OnlyEnforceIf(estacionamento.recebe_aviao_grande.Not())

In [70]:
def limita_vizinhos(modelo, estacionamentos, avioes):
  for estacionamento in estacionamentos:
    if not estacionamento.grande:
      continue
    for vizinho in estacionamento.vizinhos:
      if vizinho.grande:
        # se o vizinho tem um aviao grande => eu nao posso receber aviao grande
        modelo.Add(estacionamento.recebe_aviao_grande == 0).OnlyEnforceIf(vizinho.recebe_aviao_grande)

In [71]:
def limitar_avioes_que_requerem_passaporte(modelo, estacionamentos, avioes):
  avioes_com_controle = [aviao for aviao in avioes if aviao.requer_controle_de_passaporte]
  estacionamentos_sem_controle = [estacionamento for estacionamento in estacionamentos if not estacionamento.tem_controle_de_passaporte]
  for estacionamento in estacionamentos_sem_controle:
    for aviao in avioes_com_controle:
      modelo.Add(estacionamento.variavel != aviao.k)

In [72]:
def prefere_avioes_com_controle_de_passaporte(modelo, estacionamentos, avioes):
  penalidades = []

  estacionamentos_com_controle = [estacionamento for estacionamento in estacionamentos if estacionamento.tem_controle_de_passaporte]
  avioes_sem_controle = [aviao for aviao in avioes if not aviao.requer_controle_de_passaporte]
  for estacionamento in estacionamentos_com_controle:
    for aviao in avioes_sem_controle:
      penalidade = modelo.NewIntVar(0, 1000, f'penalidade_{estacionamento.k}_{aviao.k}')

      aviao_esta_nesse_estacionamento = modelo.NewBoolVar(f'estacionamento_{estacionamento.k}_tem_aviao_{aviao.k}')
      modelo.Add(estacionamento.variavel == aviao.k).OnlyEnforceIf(aviao_esta_nesse_estacionamento)
      modelo.Add(estacionamento.variavel != aviao.k).OnlyEnforceIf(aviao_esta_nesse_estacionamento.Not())

      modelo.Add(penalidade == 1000).OnlyEnforceIf(aviao_esta_nesse_estacionamento)
      modelo.Add(penalidade == 0).OnlyEnforceIf(aviao_esta_nesse_estacionamento.Not())

      loga(penalidade)
      penalidades.append(penalidade)
  return penalidades


In [73]:
avioes = [Aviao(1, False, True),
          Aviao(2, False, False)]
modelo = cp_model.CpModel()
total_de_avioes = len(avioes)
variaveis_a_logar = []

estacionamentos = [Estacionamento(1, total_de_avioes, False, modelo, True),
                   Estacionamento(2, total_de_avioes, False, modelo, False)]

avioes_distintos(estacionamentos, modelo)
todo_aviao_tem_que_estacionar(total_de_avioes, estacionamentos, modelo)
limita_vizinhos(modelo, estacionamentos, avioes)
limita_aviao_grande_para_estacionamento_grande(modelo, estacionamentos, avioes)
limitar_avioes_que_requerem_passaporte(modelo, estacionamentos, avioes)
penalidades= prefere_avioes_com_controle_de_passaporte(modelo, estacionamentos, avioes)

solucionador = cp_model.CpSolver()
resolve(solucionador, modelo, estacionamentos, avioes, penalidades)

OPTIMAL
Valor objetivo = 0.0
penalidade_1_2 tem valor 0
estacionamento_1 tem aviao 1 grande=False
estacionamento_2 tem aviao 2 grande=False


In [74]:
avioes = [Aviao(1, False, False),
          Aviao(2, False, True)]
modelo = cp_model.CpModel()
total_de_avioes = len(avioes)
variaveis_a_logar = []

estacionamentos = [Estacionamento(1, total_de_avioes, False, modelo, True),
                   Estacionamento(2, total_de_avioes, False, modelo, False)]

avioes_distintos(estacionamentos, modelo)
todo_aviao_tem_que_estacionar(total_de_avioes, estacionamentos, modelo)
limita_vizinhos(modelo, estacionamentos, avioes)
limita_aviao_grande_para_estacionamento_grande(modelo, estacionamentos, avioes)
limitar_avioes_que_requerem_passaporte(modelo, estacionamentos, avioes)
penalidades= prefere_avioes_com_controle_de_passaporte(modelo, estacionamentos, avioes)


solucionador = cp_model.CpSolver()
resolve(solucionador, modelo, estacionamentos, avioes, penalidades)

OPTIMAL
Valor objetivo = 0.0
penalidade_1_1 tem valor 0
estacionamento_1 tem aviao 2 grande=False
estacionamento_2 tem aviao 1 grande=False


In [75]:
avioes = [Aviao(1, False, True),
          Aviao(2, False, True)]
modelo = cp_model.CpModel()
total_de_avioes = len(avioes)
variaveis_a_logar = []

estacionamentos = [Estacionamento(1, total_de_avioes, False, modelo, True),
                   Estacionamento(2, total_de_avioes, False, modelo, False)]

avioes_distintos(estacionamentos, modelo)
todo_aviao_tem_que_estacionar(total_de_avioes, estacionamentos, modelo)
limita_vizinhos(modelo, estacionamentos, avioes)
limita_aviao_grande_para_estacionamento_grande(modelo, estacionamentos, avioes)
limitar_avioes_que_requerem_passaporte(modelo, estacionamentos, avioes)
penalidades= prefere_avioes_com_controle_de_passaporte(modelo, estacionamentos, avioes)

solucionador = cp_model.CpSolver()
resolve(solucionador, modelo, estacionamentos, avioes, penalidades)

INFEASIBLE
Sem solucao


In [76]:
avioes = [Aviao(1, False, True),
          Aviao(2, False, False),
          Aviao(3, False, False)]
modelo = cp_model.CpModel()
total_de_avioes = len(avioes)
variaveis_a_logar = []

estacionamentos = [Estacionamento(1, total_de_avioes, False, modelo, True),
                   Estacionamento(2, total_de_avioes, False, modelo, False),
                   Estacionamento(3, total_de_avioes, False, modelo, False)]

avioes_distintos(estacionamentos, modelo)
todo_aviao_tem_que_estacionar(total_de_avioes, estacionamentos, modelo)
limita_vizinhos(modelo, estacionamentos, avioes)
limita_aviao_grande_para_estacionamento_grande(modelo, estacionamentos, avioes)
limitar_avioes_que_requerem_passaporte(modelo, estacionamentos, avioes)
penalidades= prefere_avioes_com_controle_de_passaporte(modelo, estacionamentos, avioes)

solucionador = cp_model.CpSolver()
resolve(solucionador, modelo, estacionamentos, avioes, penalidades)

OPTIMAL
Valor objetivo = 0.0
penalidade_1_2 tem valor 0
penalidade_1_3 tem valor 0
estacionamento_1 tem aviao 1 grande=False
estacionamento_2 tem aviao 3 grande=False
estacionamento_3 tem aviao 2 grande=False


In [77]:
avioes = [Aviao(1, False, True),
          Aviao(2, True, False),
          Aviao(3, False, False)]
modelo = cp_model.CpModel()
total_de_avioes = len(avioes)
variaveis_a_logar = []

estacionamentos = [Estacionamento(1, total_de_avioes, False, modelo, True),
                   Estacionamento(2, total_de_avioes, False, modelo, False),
                   Estacionamento(3, total_de_avioes, True, modelo, False)]

avioes_distintos(estacionamentos, modelo)
todo_aviao_tem_que_estacionar(total_de_avioes, estacionamentos, modelo)
limita_vizinhos(modelo, estacionamentos, avioes)
limita_aviao_grande_para_estacionamento_grande(modelo, estacionamentos, avioes)
limitar_avioes_que_requerem_passaporte(modelo, estacionamentos, avioes)
penalidades= prefere_avioes_com_controle_de_passaporte(modelo, estacionamentos, avioes)

solucionador = cp_model.CpSolver()
resolve(solucionador, modelo, estacionamentos, avioes, penalidades)

OPTIMAL
Valor objetivo = 0.0
penalidade_1_2 tem valor 0
penalidade_1_3 tem valor 0
estacionamento_1 tem aviao 1 grande=False
estacionamento_2 tem aviao 3 grande=False
estacionamento_3 tem aviao 2 grande=True


In [78]:
avioes = [Aviao(1, False, True),
          Aviao(2, True, False),
          Aviao(3, False, False),
          Aviao(4, True, True)]
modelo = cp_model.CpModel()
total_de_avioes = len(avioes)
variaveis_a_logar = []

estacionamentos = [Estacionamento(1, total_de_avioes, False, modelo, True),
                   Estacionamento(2, total_de_avioes, False, modelo, False),
                   Estacionamento(3, total_de_avioes, True, modelo, False),
                   Estacionamento(4, total_de_avioes, True, modelo, True)]

avioes_distintos(estacionamentos, modelo)
todo_aviao_tem_que_estacionar(total_de_avioes, estacionamentos, modelo)
limita_vizinhos(modelo, estacionamentos, avioes)
limita_aviao_grande_para_estacionamento_grande(modelo, estacionamentos, avioes)
limitar_avioes_que_requerem_passaporte(modelo, estacionamentos, avioes)
penalidades= prefere_avioes_com_controle_de_passaporte(modelo, estacionamentos, avioes)

solucionador = cp_model.CpSolver()
resolve(solucionador, modelo, estacionamentos, avioes, penalidades)

OPTIMAL
Valor objetivo = 0.0
penalidade_1_2 tem valor 0
penalidade_1_3 tem valor 0
penalidade_4_2 tem valor 0
penalidade_4_3 tem valor 0
estacionamento_1 tem aviao 1 grande=False
estacionamento_2 tem aviao 3 grande=False
estacionamento_3 tem aviao 2 grande=True
estacionamento_4 tem aviao 4 grande=True


In [79]:
avioes = [Aviao(1, False, False),
          Aviao(2, False, False)]
modelo = cp_model.CpModel()
total_de_avioes = len(avioes)
variaveis_a_logar = []

estacionamentos = [Estacionamento(1, total_de_avioes, False, modelo, True),
                   Estacionamento(2, total_de_avioes, False, modelo, False)]

avioes_distintos(estacionamentos, modelo)
todo_aviao_tem_que_estacionar(total_de_avioes, estacionamentos, modelo)
limita_vizinhos(modelo, estacionamentos, avioes)
limita_aviao_grande_para_estacionamento_grande(modelo, estacionamentos, avioes)
limitar_avioes_que_requerem_passaporte(modelo, estacionamentos, avioes)
penalidades= prefere_avioes_com_controle_de_passaporte(modelo, estacionamentos, avioes)

solucionador = cp_model.CpSolver()
resolve(solucionador, modelo, estacionamentos, avioes, penalidades)

OPTIMAL
Valor objetivo = 1000.0
penalidade_1_1 tem valor 0
penalidade_1_2 tem valor 1000
estacionamento_1 tem aviao 2 grande=False
estacionamento_2 tem aviao 1 grande=False


In [80]:
avioes = [Aviao(1, False, False),
          Aviao(2, False, False),
          Aviao(3, False, False)]
modelo = cp_model.CpModel()
total_de_avioes = len(avioes)
variaveis_a_logar = []

estacionamentos = [Estacionamento(1, total_de_avioes, False, modelo, True),
                   Estacionamento(2, total_de_avioes, False, modelo, False),
                   Estacionamento(3, total_de_avioes, False, modelo, True)]

avioes_distintos(estacionamentos, modelo)
todo_aviao_tem_que_estacionar(total_de_avioes, estacionamentos, modelo)
limita_vizinhos(modelo, estacionamentos, avioes)
limita_aviao_grande_para_estacionamento_grande(modelo, estacionamentos, avioes)
limitar_avioes_que_requerem_passaporte(modelo, estacionamentos, avioes)
penalidades= prefere_avioes_com_controle_de_passaporte(modelo, estacionamentos, avioes)

solucionador = cp_model.CpSolver()
resolve(solucionador, modelo, estacionamentos, avioes, penalidades)

OPTIMAL
Valor objetivo = 2000.0
penalidade_1_1 tem valor 0
penalidade_1_2 tem valor 1000
penalidade_1_3 tem valor 0
penalidade_3_1 tem valor 0
penalidade_3_2 tem valor 0
penalidade_3_3 tem valor 1000
estacionamento_1 tem aviao 2 grande=False
estacionamento_2 tem aviao 1 grande=False
estacionamento_3 tem aviao 3 grande=False


In [81]:
avioes = [Aviao(1, False, False),
          Aviao(2, False, False),
          Aviao(3, False, False),
          Aviao(4, False, True)]
modelo = cp_model.CpModel()
total_de_avioes = len(avioes)
variaveis_a_logar = []

estacionamentos = [Estacionamento(1, total_de_avioes, False, modelo, True),
                   Estacionamento(2, total_de_avioes, False, modelo, False),
                   Estacionamento(3, total_de_avioes, False, modelo, False),
                   Estacionamento(4, total_de_avioes, False, modelo, False)]

avioes_distintos(estacionamentos, modelo)
todo_aviao_tem_que_estacionar(total_de_avioes, estacionamentos, modelo)
limita_vizinhos(modelo, estacionamentos, avioes)
limita_aviao_grande_para_estacionamento_grande(modelo, estacionamentos, avioes)
limitar_avioes_que_requerem_passaporte(modelo, estacionamentos, avioes)
penalidades= prefere_avioes_com_controle_de_passaporte(modelo, estacionamentos, avioes)

solucionador = cp_model.CpSolver()
resolve(solucionador, modelo, estacionamentos, avioes, penalidades)

OPTIMAL
Valor objetivo = 0.0
penalidade_1_1 tem valor 0
penalidade_1_2 tem valor 0
penalidade_1_3 tem valor 0
estacionamento_1 tem aviao 4 grande=False
estacionamento_2 tem aviao 2 grande=False
estacionamento_3 tem aviao 3 grande=False
estacionamento_4 tem aviao 1 grande=False
